### 1. How many pubs are located in each country?

```sql
SELECT
    country,
    COUNT(*) AS Total_pubs
FROM
    pubs
GROUP BY
    country;
```

### 2. What is the total sales amount for each pub, including the beverage price and quantity sold?

```sql
SELECT
    p.pub_name,
    SUM(s.quantity * b.price_per_unit) AS Total_SalesAmt
FROM
    pubs p
INNER JOIN
    sales s ON p.pub_id = s.pub_id
INNER JOIN
    beverages b ON b.beverage_id = s.beverage_id
GROUP BY
    p.pub_name;
```

### 3. Which pub has the highest average rating?

```sql
SELECT TOP 1
    p.pub_id,
    ROUND(AVG(r.rating), 2) AS Highest_avg_rating
FROM
    ratings r
INNER JOIN
    pubs p ON r.pub_id = p.pub_id
GROUP BY
    p.pub_id
ORDER BY
    2 DESC;
-- The Red Lion pub has the highest rating of 4.67
```

### 4. What are the top 5 beverages by sales quantity across all pubs?

```sql
WITH cte_rank AS (
    SELECT
        beverage_id,
        SUM(quantity) AS total_quantity,
        ROW_NUMBER() OVER (ORDER BY SUM(quantity) DESC) AS rnum
    FROM
        sales
    GROUP BY
        beverage_id
)
SELECT
    b.beverage_name AS [Top 5 beverages]
FROM
    cte_rank c
INNER JOIN
    beverages b ON c.beverage_id = b.beverage_id
WHERE
    rnum <= 5;
```

### 5. How many sales transactions occurred on each date?

```sql
SELECT
    transaction_date,
    COUNT(*) AS Sales_Frequency
FROM
    sales
GROUP BY
    transaction_date
ORDER BY
    1;
```

### 6. Find the name of someone that had cocktails and in which pub.

```sql
SELECT
    p.pub_name,
    r.customer_name
FROM
    pubs p
JOIN
    ratings r ON p.pub_id = r.pub_id
JOIN
    sales s ON s.pub_id = r.pub_id
JOIN
    beverages b ON b.beverage_id = s.beverage_id
WHERE
    b.category = 'Cocktail'
ORDER BY
    1;
```

### 7. What is the average price per unit for each category of beverages, excluding the category 'Spirit'?

```sql
SELECT
    category AS Beverage_category,
    AVG(price_per_unit) AS average_price_per_unit
FROM
    beverages
WHERE
    category <> 'Spirit'
GROUP BY
    category;
```

### 8. Which pubs have a rating higher than the average rating of all pubs?

```sql
WITH cte_ratings AS (
    SELECT
        Pub_id,
        rating,
        DENSE_RANK() OVER (PARTITION BY Pub_id ORDER BY rating DESC) AS dRank
    FROM
        ratings
    WHERE
        rating > (SELECT AVG(rating) FROM ratings)
)
SELECT
    p.*,
    c.rating
FROM
    cte_ratings c
INNER JOIN
    pubs p ON c.pub_id = p.pub_id
WHERE
    dRank = 1;
-- Filtered by only rank 1 to exclude duplicate records
```

### 9. What is the running total of sales amount for each pub, ordered by the transaction date?

```sql
WITH Cte_SalesAmt AS (
    SELECT
        s.pub_id,
        (s.quantity * b.price_per_unit) AS SalesAmt,
        s.transaction_date
    FROM
        sales s
    JOIN
        beverages b ON s.beverage_id = b.beverage_id
)
SELECT
    *,
    SUM(SalesAmt) OVER (PARTITION BY pub_id ORDER BY transaction_date) AS Running_Total_SalesAmt
FROM
    Cte_SalesAmt;
```

### 10. For each country, what is the average price per unit of beverages in each category, and what is the overall average price per unit of beverages across all categories?

```sql
WITH cte_avg_price AS (
    SELECT
        p.country,
        b.category,
        AVG(b.price_per_unit) AS [Average_price]
    FROM
        pubs p
    INNER JOIN
        sales s ON s.pub_id = p.pub_id
    INNER JOIN
        beverages b ON b.beverage_id = s.beverage_id
    GROUP BY
        p.country,
        b.category
),
cte_total_avg_price AS (
    SELECT
        p.country,
        AVG(b.price_per_unit) AS [Total_Average_price]
    FROM
        pubs p
    INNER JOIN
        sales s ON s.pub_id = p.pub_id
    INNER JOIN
        beverages b ON b.beverage_id = s.beverage_id
    GROUP BY
        p.country
)
SELECT
    a.country AS Country,
    a.category AS category,
    a.Average_price AS

 [Avg_price_per_unit],
    b.Total_Average_price AS [Total_Average_price_unit]
FROM
    cte_avg_price a
INNER JOIN
    cte_total_avg_price b ON a.country = b.country;
```

### Key Insights:

- The Red Lion Pub has the highest customer average rating of 4.67.
- The Red Lion Pub has sold the highest quantity of 55 drinks and also has generated the highest sales amount of $532.66.
- The overall average price per unit of beverages across all categories is highest for the United Kingdom and lowest for Ireland.
- The average price per unit of beverages is the highest for Whiskey in Ireland and the United Kingdom. And lowest for Beer in the United States.
- The highest transactions were done on Tuesday and Wednesday, and the lowest transaction was done on Thursday and Friday.